In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [67]:
usersDataTrain = pd.read_csv('../setEntrenamiento.csv')

In [68]:
usersDataTrain.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution_A,screen_resolution_B,...,color_Cinza espacial,new_15lastdays,sku,cantidad_De_Compras_De_Mismo_Sku,Cantidad_De_Busquedas_Por_Termino,search_and_purchase,cantidad_de_Url_de_compra,cantidadDeStaticPage,sku_multiple,label
0,0,0008ed71,3,3.0,0.0,0.0,0.0,1,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0
1,1,000c79fe,1,1.0,0.0,1.0,0.0,1,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,79.0,0
2,2,001802e4,1,1.0,0.0,5.0,0.0,1,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,24.0,0
3,3,0019e639,21,15.0,2.0,29.0,0.0,1,0.0,2.0,...,0.0,0.0,508.0,2.0,22.0,1.0,29.0,0.0,608.0,0
4,4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0.0,0.0,...,0.0,0.0,12788.0,1.0,0.0,0.0,0.0,0.0,24.0,0


In [69]:
total_train= usersDataTrain.drop( ['person','Unnamed: 0',"marca_lenovo","marca_lg","marca_sony","marca_asus","event_lead","otros","Brasilero",'Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse',
       'event_ad campaign hit', 'skus', 'marca_apple', 'marca_motorola', 'marca_samsung',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB','Cantidad_De_Busquedas_Por_Termino','cantidad_de_Url_de_compra','sku_multiple'],axis=1)

In [70]:
total_train.columns


Index(['cant_seciones', 'event_checkout', 'event_conversion',
       'screen_resolution_A', 'screen_resolution_B', 'screen_resolution_C',
       'screen_resolution_D', 'screen_resolution_E',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'ventas_region',
       'count_activity', 'conversiones_15lastdays', 'checkout_15lastdays',
       'viewed_15lastday', 'color_Preto', 'color_Dourado', 'color_Branco',
       'color_Cinza espacial', 'new_15lastdays', 'sku',
       'cantidad_De_Compras_De_Mismo_Sku', 'search_and_purchase',
       'cantidadDeStaticPage', 'label'],
      dtype='object')

In [71]:
users1s = total_train[total_train["label"] == 1]
users0s = total_train[total_train["label"] == 0].head(980)
total_train = pd.concat([users0s,users1s], ignore_index=True)

total_train["label"].value_counts()

1    980
0    980
Name: label, dtype: int64

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_train.drop(['label'], axis=1).values, total_train["label"] , test_size=0.20, random_state=75)



In [73]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 30)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 7)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [74]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.metrics import make_scorer, roc_auc_score
auc = make_scorer(roc_auc_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, re

In [75]:
rf_random.best_params_

{'n_estimators': 310,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': True}

In [76]:

clf = RandomForestClassifier(bootstrap= True,
 max_depth= 10,
 max_features =  'auto',
 min_samples_leaf=4,
 min_samples_split= 5,
 n_estimators= 1000)

In [77]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [78]:
result = clf.predict_proba(X_test)

In [79]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [80]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.88503397818106078

In [81]:
test = pd.read_csv('../setTest.csv')

In [82]:
person = test["person"]

In [83]:
test.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution_A,screen_resolution_B,...,color_Branco,color_Cinza espacial,new_15lastdays,sku,cantidad_De_Compras_De_Mismo_Sku,Cantidad_De_Busquedas_Por_Termino,search_and_purchase,cantidad_de_Url_de_compra,cantidadDeStaticPage,sku_multiple
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,0.0,0.0,...,0.0,86.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,326.0
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,53.0
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0.0,0.0,...,122.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,5.0,240.0
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,0.0,0.0,...,0.0,24.0,1.0,0.0,0.0,0.0,0.0,14.0,0.0,132.0


In [84]:
test = test.drop( ['person','Unnamed: 0',"marca_lenovo","marca_lg","marca_sony","marca_asus","event_lead","otros","Brasilero",'Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse',
       'event_ad campaign hit', 'skus', 'marca_apple', 'marca_motorola', 'marca_samsung',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB','Cantidad_De_Busquedas_Por_Termino','cantidad_de_Url_de_compra','sku_multiple'],axis=1)

In [85]:
test.head()

,cant_seciones,event_checkout,event_conversion,screen_resolution_A,screen_resolution_B,screen_resolution_C,screen_resolution_D,screen_resolution_E,Busca_terminos_populares,Computer,...,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,sku,cantidad_De_Compras_De_Mismo_Sku,search_and_purchase,cantidadDeStaticPage
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,96,7.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.0,...,96.0,2.0,130.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0
2,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,...,13.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,...,465.0,204.0,32.0,122.0,0.0,0.0,0.0,0.0,0.0,5.0
4,20,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0.0,...,177.0,3.0,64.0,0.0,24.0,1.0,0.0,0.0,0.0,0.0


In [86]:
test.columns

Index(['cant_seciones', 'event_checkout', 'event_conversion',
       'screen_resolution_A', 'screen_resolution_B', 'screen_resolution_C',
       'screen_resolution_D', 'screen_resolution_E',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'ventas_region',
       'count_activity', 'conversiones_15lastdays', 'checkout_15lastdays',
       'viewed_15lastday', 'color_Preto', 'color_Dourado', 'color_Branco',
       'color_Cinza espacial', 'new_15lastdays', 'sku',
       'cantidad_De_Compras_De_Mismo_Sku', 'search_and_purchase',
       'cantidadDeStaticPage'],
      dtype='object')

In [87]:
test.shape

(19415, 25)

In [88]:
nombreCol = []
for namecolumn in usersDataTrain.columns:
     if ( not namecolumn in test.columns):
            nombreCol.append(namecolumn)
print(nombreCol)
for namecolumn in test.columns:
     if ( not namecolumn in usersDataTrain.columns):
            nombreCol.append(namecolumn)
print(nombreCol)

['Unnamed: 0', 'person', 'event_ad campaign hit', 'event_lead', 'skus', 'otros', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg', 'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente', 'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5', 'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s', 'version_iphone 6s', 'version_moto g4', 'version_moto g2', 'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'campaign_source_google', 'campaign_source_criteo', 'campaign_source_rtbhouse', 'channel_Direct', 'channel_Paid', 'channel_Organic', 'channel_Referral', 'Brasilero', 'Cantidad_De_Busquedas_Por_Termino', 'cantidad_de_Url_de_compra', 'sku_multiple', 'label']
['Unnamed: 0', 'person', 'event_ad campaign hit', '

In [89]:
result = clf.predict_proba(test)

In [90]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [91]:
person = person.to_frame()

In [92]:
person['label'] = result2

In [93]:
person.head()

,person,label
0,4886f805,0.031984
1,0297fc1e,0.626403
2,2d681dd8,0.135348
3,cccea85e,0.695409
4,4c8a8b93,0.646612


In [94]:
(person["label"]<0.5).value_counts()

True     13849
False     5566
Name: label, dtype: int64

In [95]:
kaggle = pd.read_csv('../data/trocafone_kaggle_test.csv')

In [96]:
kaggle.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [97]:
resultado = pd.merge(kaggle,person,on="person",how = "left")

In [98]:
resultado.head()

,person,label
0,4886f805,0.031984
1,0297fc1e,0.626403
2,2d681dd8,0.135348
3,cccea85e,0.695409
4,4c8a8b93,0.646612


In [99]:
resultado["label"].isnull().value_counts()

False    19415
Name: label, dtype: int64

In [100]:
resultado.to_csv('resultadoRandomForestUnderSamplingUltimo.csv', index=False)